In [1]:
from pandas import read_excel
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
def load_csv(filename):
    file = open(filename, "r", encoding="utf-8")
    experiments = []
    headers = []
    reader = csv.reader(file)
    for i, lines in enumerate(reader):
        if i == 0:
            #headers = [e for e in lines[0].split(";")]
            h = lines[0].split(";")
            headers.append(h)
        else:
            tmp = lines[0].split(";")
            for j in range(0,len(tmp)):
                if (j != 0 and j != 1879):
                    tmp[j] = float(tmp[j])
                else:
                    continue
            #tmp = [float(tmp[j]) for j in range(0,len(tmp)) if (j!=0 and j!=1879)] #This removes two entries, dunno why. 
            experiments.append(tmp)
    file.close()
    return headers, experiments

def column_extraction(data_list,column_number): #columns start counting in 0
    col = []
    for i in range(0,len(experiments)):
        dato = experiments[i][column_number]
        col.append(dato)
    return col   

## Load both data sets for comparision

In [3]:
# Loading compound library
my_sheet = 'SPECS_drug_repurp_set' # change it to your sheet name, you can find your sheet name at the bottom left of your excel file
file_name = r'/home/jovyan/covid-data/SPECS_drug_repurp_set.xlsx'# change it to the name of your excel file
df = read_excel(file_name, sheet_name = my_sheet)
#print(df.head()) # shows headers with top 5 rows
library_compound_ids = df[["Compound ID"]].copy() #Extracting ID's form specs
library_compound_ids.rename(columns = {'Compound ID':'ID_library'}, inplace = True)

In [4]:
#Loading Maris' data
headers, experiments = load_csv('/home/jovyan/covid-data/dalia_MeanFeaturesPerWell_BatchA.csv')
#Column 1879 contains the compound ID
screening_compound_ids = pd.DataFrame(column_extraction(experiments,1879), columns=['ID_Screening'])#Extracting ID's form covid

In [5]:
print(f'The library has {len(library_compound_ids)} compounds')
print(f'The experiment has {len(screening_compound_ids)} compounds')

The library has 5280 compounds
The experiment has 6065 compounds


### Analysing SPECS compound repetitions

In [6]:
# Removing compunds with ID "Problem with structure"
problem = library_compound_ids.loc[library_compound_ids['ID_library']=='Problem with structure']

In [7]:
#Check that we did not miss information
library_compound_ids_2 = library_compound_ids.copy().drop([3471, 5185])
print(f'The library has now {len(library_compound_ids_2)} compounds')

The library has now 5278 compounds


In [8]:
# Finding repetitions
library_compound_ids_2 = library_compound_ids_2.copy().groupby('ID_library').size().sort_values(ascending=False).to_frame() #Count the elements in each group and sort
library_compound_ids_2.rename(columns = {0:'Count'}, inplace = True)
# Showing non-unique compounds
unique_id_lib_count = library_compound_ids_2.copy().reset_index()
a = unique_id_lib_count[unique_id_lib_count['Count'] >1]
print(f'Printing repeated compounds {a}')
repeated_compounds = unique_id_lib_count['ID_library'][unique_id_lib_count['Count'] >1].tolist() #Storing only repeated compounds
repeated_compounds.remove('CBK277930') # <------------------OUTLIER COMPOUND

Printing repeated compounds    ID_library  Count
0   CBK011558      3
1   CBK011603      3
2   CBK042067      3
3   CBK041250      3
4   CBK000879      3
..        ...    ...
78  CBK200659      2
79  CBK041357      2
80  CBK041430      2
81  CBK041511      2
82  CBK309105      2

[83 rows x 2 columns]


In [9]:
specs = df.copy().drop([3471, 5185])
experiments_df = pd.DataFrame(experiments , columns=headers[0])
specs.head()
#experiments_df['CompoundID'].head()

,Library,Compound ID,Batch nr,CUSTOMER_ID,MOLFORMULA,MOLWEIGHT,NAME,VENDOR,ADD_INFO,SMILES,IUPAC_NAME,STEREOCHEMISTRY,pert_iname,clinical_phase,moa,target,disease_area,indication
0,SPECS,CBK309065,BJ1897547,A00047421,C46H47ClN8O9S,923.45,ARV-825,MedChemExpress,"CAS 1818885-28-7, target Epigenetic Reader Dom...",CC1=NN=C2N1C(SC(C)=C3C)=C3C(C4=CC=C(Cl)C=C4)=N...,"2-[(9S)-7-(4-chlorophenyl)-4,5,13-trimethyl-3-...",(9S)-isomer,ARV-825,Preclinical,bromodomain inhibitor,BRD4,NaN,NaN
1,SPECS,CBK041145,BJ1894313,A00077618,C10H10BrN5O7P.Na.H2O,464.11,8-bromo-cGMP,Bio-Techne,CAS 51116-01-9,[H][C@@]12COP([O-])(=O)O[C@@]1([H])[C@@H](O)[C...,"8-Bromoguanosine cyclic 3',5'-monophosphate so...",NaN,8-bromo-cGMP,Preclinical,PKA activator,PRKG1,NaN,NaN
2,SPECS,CBK200894,BJ1898003,A00100033,C10H13N3O5S,287.30,nifurtimox,Prestwick Chemical,NaN,CC1CS(=O)(=O)CCN1N=Cc1ccc(o1)[N+]([O-])=O,"(E)-N-(3-methyl-1,1-dioxo-1,4-thiazinan-4-yl)-...",E-isomer,nifurtimox,Launched,DNA inhibitor,NaN,infectious disease,Chagas disease|African trypanosomiasis
3,SPECS,CBK309015,BJ1897487,A00147595,C20H17N3O4S,395.44,Balaglitazone,MedChemExpress,"CAS 199113-98-9, target PPAR",O=C(N1)SC(CC2=CC=C(OCC(N3C)=NC4=C(C=CC=C4)C3=O...,5-[[4-[(3-methyl-4-oxo-quinazolin-2-yl)methoxy...,NaN,balaglitazone,Phase 3,insulin sensitizer|PPAR receptor partial agonist,PPARG,NaN,NaN
4,SPECS,CBK041791,BJ1896935,A00218260,C22H16F2N2,346.38,Flutrimazole,Sigma-Aldrich,CAS 119006-77-8,Fc1ccc(cc1)C(c2ccccc2)(c3ccccc3F)n4ccnc4,1-[(2-fluorophenyl)-(4-fluorophenyl)-phenyl-me...,NaN,flutrimazole,Launched,sterol demethylase inhibitor,NaN,infectious disease,mycosis


In [10]:
experiments_df.head()

,PlateWellCompound,Count_nuclei,AreaShape_Area_nuclei,AreaShape_BoundingBoxArea_nuclei,AreaShape_BoundingBoxMaximum_X_nuclei,AreaShape_BoundingBoxMaximum_Y_nuclei,AreaShape_BoundingBoxMinimum_X_nuclei,AreaShape_BoundingBoxMinimum_Y_nuclei,AreaShape_Center_X_nuclei,AreaShape_Center_Y_nuclei,...,RadialDistribution_ZernikePhase_illumSYTO_8_8_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_9_1_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_9_3_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_9_5_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_9_7_cytoplasm,RadialDistribution_ZernikePhase_illumSYTO_9_9_cytoplasm,CompoundID,principal component 1,Ypredicted,Intensity_MeanIntensity_illumMITO_cells
0,001AA24Uninfected,137.857143,2042.971644,2697.331820,1049.009917,1126.432454,997.929124,1074.662577,1022.985229,1100.039830,...,0.001906,0.058730,0.048248,0.054010,0.024222,0.017357,Uninfected,-76.289086,0.920807,0.006205
1,001AB24Uninfected,138.666667,2007.938584,2636.214152,1026.251972,1031.763719,975.189540,981.030930,1000.237019,1005.876806,...,0.012006,0.063354,-0.021407,-0.013524,0.040170,-0.050091,Uninfected,-78.554542,0.947901,0.006724
2,001AC24Uninfected,130.777778,2018.087616,2645.133704,1165.492031,1116.441738,1114.331548,1065.679890,1139.405329,1090.561927,...,0.050027,0.033613,0.031683,-0.031786,-0.024306,0.007508,Uninfected,-77.122410,0.954721,0.005685
3,001AD24Uninfected,121.666667,1997.428138,2623.199160,1120.647033,1110.348499,1070.125740,1059.305848,1094.882670,1084.309388,...,0.074355,0.073954,0.024698,-0.159391,-0.089118,0.059125,Uninfected,-77.669307,0.959160,0.004975
4,001AE24Uninfected,135.333333,2031.776072,2662.351290,1064.428878,1167.848794,1013.334589,1116.671344,1038.389049,1141.806274,...,0.015527,-0.014393,-0.053013,-0.011721,-0.104811,-0.007278,Uninfected,-77.621119,0.963378,0.004517


In [11]:
counter = 0
median_repeated_compounds = []
median_repeated_compounds_idx = []
for compound in repeated_compounds:
    
    np.random.seed(17)
    
    aux = experiments_df['Intensity_MeanIntensity_illumMITO_cells'].loc[experiments_df['CompoundID'] == compound] # Accessing info from the repeated compounds found in SPECS
    aux.columns = ['0','1'] #Renaming columns bcs of problem with column replication
    aux.drop('0', axis=1, inplace=True)
    aux2 = aux.sort_values(by=['1'], ascending=True) # Sort to access the mean
    
    # Non-permanent solution
    if len(aux2) == 2:
        median_index = np.random.choice(aux.index.tolist())
        median = aux2['1'][median_index]
    else:
        median = aux2.median(axis=0).tolist()
        median_index = aux2.index[aux2['1'] == median[0]].tolist()
        median_index = median_index[0]
    #print(median_index)
    #print(median)
    
    median_repeated_compounds.append(median)
    median_repeated_compounds_idx.append(median_index)
    counter += 1
    #if counter == 1:
    #    break

## Create the data sets to use 

In [12]:
len(median_repeated_compounds_idx)

82